In [10]:
###CODE WRITTEN BY ZACHARY SORENSON###

import numpy as np
from numpy import *
import scipy as sp
import pandas as pd
import matplotlib as mpl
import sys
import math
import re
from enum import Enum
from itertools import cycle, islice, dropwhile
import random
import time
import statistics
import tsplib95
import networkx

# NOTE: Compile the last block of code before running any subroutines

# Executable

In [ ]:
# load the tsplib problem (if file is .tsp or .atsp) ###
# ELSE skip to INSERT
# problem = tsplib95.load('pr76.tsp')


# # convert into a networkx.Graph
# down = problem.get_graph()

# # convert into a numpy distance matrix
# hold1 = networkx.to_numpy_matrix(down)
# hold2 = np.array(hold1)

# datre=Dread(hold2)

# END .tsp/.atsp Loader

#INSERT title of data set here (IF USING .TXT)

holddat=np.array(pd.read_csv('22cities.txt', header=None, sep='\s+', engine='python'))
datre=Dread(holddat)

#END .txt Loader


#Insert optimal, or best known solution, in for "checkopt"
#NOTE for testing purposes, I created a function "dataopt" that stores the optimal solutions for ou
checkopt=dataopt(len(datre))
print("optimal",checkopt)


#Insert desired number of trials in for "rarnge"
rarnge=10
#Insert desired Opt algorithm. "0" for 2-Opt, "1" for 3-Opt, "2" for k-Opt
typer=2
#Insert "1" for randomized tours, "0" for standard tour of (0,1,2,...)
rando=1
#Insert "1" to compare results for Opt+CCP+Opt and "0" for Opt+CCP
ktyper=1
#Insert "0" for success rate computation and "1" for improvement amount
surim=1

    
del time
import time

#NOTE THIS CODE COMPARES OPT SWITCH TO ORIGINAL TOUR

if surim == 0:

    success = SuccessRate(checkopt,datre,rarnge,typer,rando,ktyper)

else:

    improvem1,improvem2,time=ImproveRateM(checkopt,datre,rarnge,typer,rando,ktyper)

if typer==0:
    print("For 2-Opt")
if typer==1:
    print("For 3-Opt")
if typer==2:
    print("For k-Opt")



if surim == 0:
    
    if ktyper==0:
        print("Success rate of CCP is",success)
    if ktyper==1:
        print("Success rate of CCP+ Opt is",success)

else:
    
    if ktyper==0:
        print("For CCP Only")
    if ktyper==1:
        print("For CCP + Opt")

    print("The relative improvement for CCP is")
    print(improvem1*100)
    print("The closure of the gap between Optimal and Found Solution for CCP is")
    print(improvem2*100)
    print("The average run time is")
    print(time,"time2")
        


#### 

# Compile This First

In [11]:
#The following function contains Optimal/Best found cost of tours
def dataopt(lenopt):
    if lenopt==12:
        return 1733
    if lenopt==22:
        return 781
    if lenopt==29:
        return 27603
    if lenopt==30:
        return 503
    if lenopt==59:
        return 1003
    if lenopt==128:
        return 19033
    if lenopt==131:
        return 564
    if lenopt==237:
        return 1019
    if lenopt==343:
        return 1368
    if lenopt==76:
        return 108159
    if lenopt==17:
        return 108159
    if lenopt==96:
        return 55209
    

#This function converts astp and tsp files into matrices  
def Dread(matri):

    int1=int(len(matri))
    int2=int(len(matri[0]))
    int3=int(int1*int2)
    msize=len(matri[0])
    ares = np.array([ 0 for i in range(int3)])
    aresp = np.array([ [ 0 for i in range(msize) ] for j in range(msize) ])
    d=0
    mo=-1
    for i in range(int1):
        for j in range(int2):
            if (np.isnan(matri[i][j])):
                d=d+1
                break
            mo=mo+1
            ares[mo]=matri[i][j]
    mo=-1
    for i in range(msize):
        for j in range(msize):
            mo=mo+1
            aresp[i][j]=ares[mo]
    return aresp
    
    

#reverses the order of the tour
def Fliptour(Tour):
    Flipped=[Tour[len(Tour)-1-i] for i in range(len(Tour))]
    return Flipped
        

def SuccessRate(opt,datsu,numtri,ktype,rands,kimp):
    #Variable initializing
    clarr=[]
    subarr=[]
    totsubs=0
    subco=0
    checkop=opt
    tt=0
    k=0
    ranger=24
    subsuccess=[0 for i in range(ranger)]
    subfail=[0 for i in range(ranger)]
    sus1=0
    sus2=0
    sus3=0
    sus4=0
    sus5=0
    sus6=0
    suf1=0
    suf2=0
    Tour4s=[i for i in range(len(datsu))]
    Tour4s1=[i for i in range(len(datsu))]
    Tour3s=[i for i in range(len(datsu))]
    Tour3s1=[i for i in range(len(datsu))]
    Tour3s1u=[i for i in range(len(datsu))]
    Tour2s=[i for i in range(len(datsu))]
    Tour2s1=[i for i in range(len(datsu))]
    Tour5u=[i for i in range(len(datsu))]
    sustore=0
    Cost2s=math.inf
    CCTour2s=math.inf
    CCTour2s1f=math.inf
    CCTour3s=math.inf
    CCTour4s=math.inf
    CCTour2s1=math.inf
    CCTour3s1=math.inf
    CCTour4s1=math.inf
    tot1=0
    
    
    #Runs through a while loop until the specified number of trials is reached
    while k != (numtri):
        start1=time.time()
        Datstoru =[]
        Datstoru = np.copy(datsu)
        routes=[]
        routes=[i for i in range(len(datsu))]
        if rands!=0:    
            random.shuffle(routes)
        
        #Initial run for Opt algorithms
        if ktype==0:
            Tour2s, Cost2s = two_opt(datsu,routes)    
        if ktype==1:
            Tour3s = tsp_3_opt(datsu,routes)
            datsu = np.copy(Datstoru)
        if ktype==2:
            Tour2u, Cost2u = two_opt(datsu,routes)
            Tour5u = tsp_3_opt(datsu,Tour2u)
            datsu = np.copy(Datstoru)
        
        #If we want to run Opt algorithms twice
        if kimp==1:
            #check to see if initial tour is optimal
            if cost(datsu,Tour3s)==checkop or cost(datsu,Tour2s)==checkop or cost(datsu,Tour5u)==checkop:
                k=k-1
            else:
                
                #If 2-Opt is the chosen option
                if ktype==0:
                    #run CCP subroutine
                    CCTour2s1,CT2s1,tt,subs,cly,choork = CCP(datsu,Tour2s)
                    #check that solution is feasible
                    if tt==1:
                        datsu = np.copy(Datstoru)
                        continue
                    datsu = np.copy(Datstoru)
                    #run Opt algorithm final time
                    Tour2s1f, Cost2s1f = two_opt(datsu,CT2s1)
                    
                #If 3-Opt is the chosen option
                if ktype==1:
                    CCTour3s1,CT3s1,tt,subs,cly,choork = CCP(datsu,Tour3s)
                    if tt==1:
                        datsu = np.copy(Datstoru)
                        continue
                    datsu = np.copy(Datstoru)
                    Tour3s1 = tsp_3_opt(datsu,CT3s1)
                    datsu = np.copy(Datstoru)
                    
                #If 2+3-Opt is the chosen option
                if ktype==2:
                    CCTour4s1,CT4s1,tt,subs,cly,choork = CCP(datsu,Tour5u)
                    if tt==1:
                        datsu = np.copy(Datstoru)
                        continue
                    datsu = np.copy(Datstoru)
                    Tour4s1u, Cost2s1u = two_opt(datsu, CT4s1)
                    Tour3s1u = tsp_3_opt(datsu,Tour4s1u)
                
                #print("new",cost(datsu,Tour3s1u),"old",cost(datsu,Tour5u))
                
                #Counters for succesful runs
                #2+C+2 < 2 Opt
                if CCTour2s1 < Cost2s:
                    sus4=sus4+1
                #3+C+3 < 3
                if cost(datsu,Tour3s1) < cost(datsu,Tour3s):
                    sus5=sus5+1
                #2+3+C+2+C < 2+3
                if cost(datsu,Tour3s1u) < cost(datsu,Tour5u):
                    sus6=sus6+1
                end1=time.time()
                #store total time for individual run
                tot1=tot1+end1-start1
                    
        #If we only want to run CCTP
        if kimp==0:
            if cost(datsu,Tour3s)==checkop or cost(datsu,Tour2s)==checkop:
                k=k-1
            else:
                datsu = np.copy(Datstoru)
                if ktype==1:
                    CCTour3s,CT3s,tt,subs,cly,choork = CCP(datsu,Tour3s)
                    if tt==1:
                        datsu = np.copy(Datstoru)
                        continue
                    datsu = np.copy(Datstoru)
                    
                if ktype==0:
                    CCTour2s,CT2s,tt,subs,cly,choork = CCP(datsu,Tour2s)
                    if tt==1:
                        datsu = np.copy(Datstoru)
                        continue
                    datsu = np.copy(Datstoru)

                if ktype==2:
                    CCTour4s,CT4s,tt,subs,cly,choork = CCP(datsu,Tour5u)
                    
                    if tt==1:
                        datsu = np.copy(Datstoru)
                        continue
                    datsu = np.copy(Datstoru)

                #CCP<2
                if CCTour2s < Cost2s:
                    sus1=sus1+1
                #CCP<3
                if CCTour3s < cost(datsu,Tour3s):
                    sus2=sus2+1 
                    sustore=1
                #CCP<2+3
                if CCTour4s < cost(datsu,Tour5u):
                    sus3=sus3+1
                    sustore=1
                
                #Used to store the number of successes for subtour amounts
                if sustore==1:
                        subsuccess[subs]+=1
                if sustore==0:
                        subfail[subs]+=1
                
                #stores number of subtours
                if choork == 1:
                    clarr.append(cly)
                    subarr.append(subs)
                    totsubs=subs+totsubs
                    subco=subco+1
                end1=time.time()
                tot1=tot1+end1-start1
                
        sustore=0            
        datsu = np.copy(Datstoru)
        k=k+1
    
    #UNCOMMENT THE FOLLOWING BLOCK IF YOU WANT TO PRINT OUT SUBTOUR NO. Vs SUCCESS RATE
#     for i in range(len(subsuccess)):
#         if subsuccess[i]!=0:
#             print("i=",i, "subsuccess",subsuccess[i])
        
#     for i in range(len(subsuccess)):
#         if subfail[i]!=0:
#             print("i=",i, "subfail",subfail[i])
            
    print("Total Time",tot1)
    
    if kimp==0:
        if ktype==0:
            return sus1
        if ktype==1:
            return sus2
        if ktype==2:
            return sus3

    if kimp==1:
        if ktype==0:
            return sus4
        if ktype==1:
            return sus5
        if ktype==2:
            return sus6
        
        
def ImproveRateM(checkopm,datsum,numtrim,ktypem,randsm,kimpm):

    #Initialize Variables
    tums1=0
    tums2=0
    tums3=0
    tums4=0
    tums5=0
    tums6=0
    trimy1=0
    trimy2=0
    k=1
    avtime1=0
    avtime2=0
    badtime=0
    besty=math.inf
    count1=0
    subs=0
    totsubs=0
    subco=0
    subarr=[]
    cly=0
    clarr=[]
    choork=0
    start=time.time()
    start3=time.time()

    
    #Run loop for requested number of trials
    while k != numtrim+2:    
        #Reinitialize variables at the beginning of every trial
        Tour4sm=[i for i in range(len(datsum))]
        Tour4s1m=[i for i in range(len(datsum))]
        Tour3sm=[i for i in range(len(datsum))]
        Tour3s1m=[i for i in range(len(datsum))]
        Tour3s1um=[i for i in range(len(datsum))]
        Tour2sm=[i for i in range(len(datsum))]
        Tour2s1m=[i for i in range(len(datsum))]
        Tour5um=[i for i in range(len(datsum))]
        Cost2sm=math.inf
        CCTour2sm=math.inf
        CCTour3sm=math.inf
        CCTour4sm=math.inf
        CCTour2s1m=math.inf
        CCTour3s1m=math.inf
        CCTour4s1m=math.inf
        Cost2s1m=math.inf
        badtime2=0
        time1=0
        time2=0
        avtime2=0
        start2=time.time()
        ums1=0
        ums2=0
        ums3=0
        ums4=0
        ums5=0
        ums6=0
        Datstorm =[]
        Datstorm = np.copy(datsum)
        routes=[]
        routes=[i for i in range(len(datsum))]
        
        if randsm!=0:    
            random.shuffle(routes)
        
        if ktypem==0:
            Tour2sm, Cost2sm = two_opt(datsum,routes)        
        
        if ktypem==1:
            Tour3sm = tsp_3_opt(datsum,routes)
            datsu = np.copy(Datstorm)
        
        if ktypem==2:
            Tour2um, Cost2um = two_opt(datsum,routes)
            Tour5um = tsp_3_opt(datsum,Tour2um)
            datsu = np.copy(Datstorm)    
                    
        if cost(datsum,Tour3sm)==checkopm or cost(datsum,Tour2sm)==checkopm or cost(datsum,Tour5um)==checkopm:
            k=k-1
            end2=time.time()
            end3=time.time()
            badtime=badtime+end2-start2
            badtime2=badtime2+end3-start2
        else:
            #Remove the time of any bad run
            count1=count1+1
            if count1 == numtrim:
                time2=time.time()-start2-badtime2
                
            if ktypem==0:
                if kimpm == 0:
                    CCTour2s1m,CT2s1m,tt,subs,cly,choork = CCP(datsum,Tour2sm)
                    datsum = np.copy(Datstorm)
                    if tt==1:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        datsum = np.copy(Datstorm)
                        continue
                    if choork == 1:
                        clarr.append(cly)
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                    else:
                        k=k-1
                    #If there is an improvement 
                    if CCTour2s1m < Cost2sm:
                        #Relative Improvement
                        ums1=(Cost2sm-CCTour2s1m)/(Cost2sm)
                        tums1=ums1+tums1
                        #Gap Closure
                        ums2=(Cost2sm-CCTour2s1m)/(Cost2sm-checkopm)
                        tums2=ums2+tums2
                    else:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        k=k-1  
                        
                else:
                    CCTour2s1m,CT2s1m,tt,subs,cly,choork = CCP(datsum,Tour2sm)
                    
                    datsum = np.copy(Datstorm)
                    if tt==1:
                        datsum = np.copy(Datstorm)
                        end2=time.time()
                        badtime=badtime+end2-start2
                        continue
                        
                    if choork == 1:
                        clarr.append(cly)
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                    else:
                        k=k-1  
                    
                        
                    Tour2s1m, Cost2s1m = two_opt(datsum,CT2s1m)
                    
                    Cost2s1m = cost(datsum,Tour2s1m)
                    

                    if Cost2s1m < Cost2sm:
                        ums1=(Cost2sm-Cost2s1m)/(Cost2s1m)
                        tums1=ums1+tums1
                        ums2=(Cost2sm-Cost2s1m)/(Cost2sm-checkopm)
                        tums2=ums2+tums2           
                    else:
                        k=k-1
                        end2=time.time()
                        badtime=badtime+end2-start2
                    
                if k==(numtrim):
                    time1=time.time()-start-badtime
                    trimy1=tums1/numtrim
                    trimy2=tums2/numtrim
                    avtime1=time1/numtrim
                    return trimy1,trimy2,avtime1
                
            if ktypem==1:
                if kimpm == 0:
                    CCTour3s1m,CT3s1m,tt,subs,cly,choork = CCP(datsum,Tour3sm)
                    datsum = np.copy(Datstorm)
                    if tt==1:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        continue
                        
                    if choork == 1:
                        clarr.append(cly)
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                    else:
                        k=k-1  
                            
                    if CCTour3s1m < cost(datsum,Tour3sm):
                        ums1=(cost(datsum,Tour3sm)-CCTour3s1m)/(cost(datsum,Tour3sm))
                        tums1=ums1+tums1
                        ums2=(cost(datsum,Tour3sm)-CCTour3s1m)/(cost(datsum,Tour3sm)-checkopm)
                        tums2=ums2+tums2
                    else:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        k=k-1         
                        
                else:
                    CCTour3s1m,CT3s1m,tt,subs,cly,choork = CCP(datsum,Tour3sm)
                    
                    datsum = np.copy(Datstorm)
                    
                    if tt==1:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        continue
                        
                    if choork == 1:
                        clarr.append(cly)
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                        
                    Tour3smu = tsp_3_opt(datsum,CT3s1m)
                    
                    if cost(datsum,Tour3smu) < cost(datsum,Tour3sm):
                        ums1=(cost(datsum,Tour3sm)-cost(datsum,Tour3smu))/(cost(datsum,Tour3sm))
                        tums1=ums1+tums1
                        ums2=(cost(datsum,Tour3sm)-cost(datsum,Tour3smu))/(cost(datsum,Tour3sm)-checkopm)
                        tums2=ums2+tums2
                    else:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        k=k-1
                              
                #After last run, return the time results   
                if k==(numtrim):
                    time1=time.time()-start-badtime
                    trimy1=tums1/numtrim
                    trimy2=tums2/numtrim
                    avtime1=time1/numtrim
                    return trimy1,trimy2,avtime1
                
            if ktypem==2:
                if kimpm == 0:
                    CCTour3s1m,CT3s1m,tt,subs,cly,choork = CCP(datsum,Tour5um)

                    datsum = np.copy(Datstorm)
                    if tt==1:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        
                        continue
                        
                    if choork == 1:
                        clarr.append(cly)
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                    else:
                        k=k-1  

                    if CCTour3s1m < cost(datsum,Tour5um):
                        ums1=(cost(datsum,Tour5um)-CCTour3s1m)/(cost(datsum,Tour5um))
                        tums1=ums1+tums1
                        ums2=(cost(datsum,Tour5um)-CCTour3s1m)/(cost(datsum,Tour5um)-checkopm)
                        tums2=ums2+tums2
                    else:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        k=k-1  
                                      
                else:
                    CCTour3s1m,CT3s1m,tt,subs, cly, choork = CCP(datsum,Tour5um)
                    datsum = np.copy(Datstorm)
                    
                    if tt==1:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        continue
                        
                    if choork == 1:
                        clarr.append(cly)
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                    else:
                        k=k-1  
                        
                    Tour2ums, Cost2ums = two_opt(datsum,CT3s1m)
                    Tour3smu = tsp_3_opt(datsum,Tour2ums)
                    datsum = np.copy(Datstorm)
                    
                    if cost(datsum,Tour3smu) < cost(datsum,Tour5um):
                        ums1=(cost(datsum,Tour5um)-cost(datsum,Tour3smu))/(cost(datsum,Tour5um))
                        tums1=ums1+tums1
                        ums2=(cost(datsum,Tour5um)-cost(datsum,Tour3smu))/(cost(datsum,Tour5um)-checkopm)
                        tums2=ums2+tums2
                    else:
                        end2=time.time()
                        badtime=badtime+end2-start2
                        k=k-1
                        
            
                if k==(numtrim):
                    time1=time.time()-start-badtime
                    trimy1=tums1/numtrim
                    trimy2=tums2/numtrim
                    avtime1=time1/numtrim
                    print("besty",besty)
                    return trimy1,trimy2,avtime1
                              
                
        if subco==numtrim:
            print("average",statistics.mean(clarr),"subco")
            print("Median", statistics.median(clarr), "Range", min(clarr),"-",max(clarr))
        datsum = np.copy(Datstorm)
        k=k+1
            
            
#breaks tour into subtours
def CycleBreakSub(ListBruk, jumpk, revk):
    #Initilize a 2-d array, one dimension for indices, the other for vertex number
    jumpystork=[[-1 for i in range(2)] for j in range(len(jumpk))]
    for i in range(len(jumpk)):
        for j in range(2):
            jumpystork[i][j]=jumpk[i][j]
    
    #initialize variables
    UpdatedLuk=[-1]
    UpdatedFun=[]
    indyx=0
    countroo=-1
    minny=math.inf
    minnystore=math.inf
    jumpystore=math.inf
    jumpnex=-1
    infcounter=0
    hhold=-1
    zerond=-1
    brucker=-1
    kstore=-1
    jumpzer=-1
    minicho=-1
    stmin=-1
    jmin=-1
    
    
    #Check to see if one of the jumps has is the first term
    for i in range(len(jumpk)):
        if jumpk[i][0] == 0:
            jumpzer=0
        
    #continue to run until negative cycle has been iterated through entirely
    #process is a little convaluted, but essentially it considers every possible case
    #that could appear for a given "edge" and prepares them for the next loop   
    while len(UpdatedLuk) != len(ListBruk):
        if UpdatedLuk[0] == -1:
            UpdatedLuk=[]
        for i in range(len(jumpk)):
            minny=revk[i][0]
            if minny < minnystore and minny>=jumpnex:
                minnystore=minny
        if (stmin==minnystore and jmin==jumpnex):
            break
        stmin=minnystore
        jmin=jumpnex
        if minnystore==jumpnex:
            if ListBruk[indyx] not in UpdatedLuk:
                UpdatedLuk.append(minnystore)
            else:
                break
        while indyx != minnystore+1:
            if indyx==len(ListBruk):
                break
            if ListBruk[indyx]==0 and zerond==0:
                brucker=0
                break
            if ListBruk[indyx]==0:
                zerond=0
            if ListBruk[indyx] not in UpdatedLuk:
                UpdatedLuk.append(ListBruk[indyx])
                countroo=countroo+1
            indyx=indyx+1
        if brucker==0 and jumpzer==-1:
            break
        for h in range(len(revk)):
            if(jumpk[h][0]==minnystore):
                jumpnex=jumpk[h][1]
                indyx=jumpnex
                jumpk[h][0]=math.inf
                infcounter=infcounter+1
        minny=math.inf
        minnystore=math.inf
    
    #reinitialize variables
    UpdatedFun.append(UpdatedLuk)
    UpdatedLuk=[]
    jumpnex=math.inf
    minny=-1
    startu=0
    startval=-1
    indyx=0
    hstore=-1
    istore=-1
    minny=math.inf
    minnystore=math.inf
    jumpy=math.inf
    jumpystore=math.inf
    jumpnex=0
    countroo=0
    infcounter=1
    finishd=0
    brek=0
    
    #This is where the CCTP sends a "flow of one" through the network
    while infcounter < len(revk):
        for h in range(len(revk)):
            if(math.isinf(jumpk[h][0])==0):
                jumpy=jumpk[h][1]
                if jumpy < jumpystore and minny>jumpnex:
                    jumpystore=jumpy
                    hstore=h
                if(revk[i][0]<minnystore and minny>=jumpnex):
                        minny=revk[i][0]
                        istore=i               
        infcounter=infcounter+1 
        for i in range(len(jumpk)):
            minny=revk[i][0]
            if minny < minnystore and minny >= jumpystore: 
                minnystore=minny
        indyx=jumpystore
        minny=math.inf
        if jumpystore != minnystore +1:       
            while countroo + jumpystore != minnystore +1:
                if indyx==minnystore+1 and UpdatedLuk:
                    if UpdatedLuk[len(UpdatedLuk)-1] != UpdatedFun[len(UpdatedFun)-1][len(UpdatedFun[len(UpdatedFun)-1])-1]:
                        UpdatedFun.append(UpdatedLuk)
                    UpdatedLuk=[]
                    break
                if indyx == len(ListBruk):
                    break
                if ListBruk[indyx] not in UpdatedLuk:
                    UpdatedLuk.append(ListBruk[indyx])
                countroo=countroo+1
                indyx=indyx+1 
            for i in range(len(jumpk)):
                if jumpystork[i][0]+1==indyx:
                    if UpdatedLuk:
                        if UpdatedLuk[0] == jumpystork[i][1]:
                            UpdatedFun.append(UpdatedLuk)
                            UpdatedLuk=[]
                            break
                            
        countroo=0   
        jumpk[hstore][0]=math.inf
        hstore=-1
        istore=-1
        countroo=countroo-1
        for h in range(len(revk)):
            if(jumpk[h][0]==minnystore):
                jumpnex=jumpk[h][1]
                indyx=jumpnex
        minny=math.inf
        minnystore=math.inf
        jumpy=math.inf
        jumpystore=math.inf     
        
    return UpdatedFun


def CCP(Datc,routcc):
    
    Datstor =[]
    Datstor = np.copy(Datc)
    coststory=[]
    Plourd=[]
    coutn=1
    coutnn=1
    costup=math.inf
    recyc=[]
    emptour=[]
    tc=0
    timemin=0
    timepatch=0
    subnumb=0
    lyc=0
    chork=0
    #Calculate only the time to run MinMean
    startmin=time.time()
    
    #Detect a negative cycle
    cycler,surm,indyroutem=minmean(Datc,routcc)
    
    #If there isn't a negative cycle, terminate
    if surm < 0:
        chork=1
    
    timemin=time.time()-startmin
    startpatch=time.time()
    lyc=len(cycler)
    recyc = cycler[::-1]
    emptour=[]
    
    #store the indices of the negative cycle
    for i in range(len(recyc)):
        for j in range(len(routcc)):
            if recyc[i]==routcc[j]:
                emptour.append(j)
                                 
    #find edges in negative cycle and subtous
    jumpc,revc=Jumps(routcc,emptour)
    Subtourc= CycleBreakSub(indyroutem[1],jumpc,revc)
    
    subnumb= len(Subtourc)
    
    #combine subtours into a single array
    Listfin=Listy(indyroutem,Subtourc)
    Datc=Datstor

    
    #Form final tour and calculate cost of final tour
    if len(Listfin) > 1:
        Plourd=Tourpr(Listfin,Datc,Subtourc)
        if costd(Datc,Plourd) not in coststory:
            coststory.append(costd(Datc,Plourd))
            coutn=coutn+1
            tc=Tourcheck(Plourd,Datc)
    else:
        if costd(Datc,Listfin[0]) not in coststory:
            coststory.append(costd(Datc,Listfin[0]))
            coutn=coutn+1
            tc=Tourcheck(Listfin[0],Datc)
            Plourd=Listfin[0]       
    
    for i in range(len(coststory)):
        if coststory[i] < costup:
            costup=coststory[i]
            
    costup=costd(Datc,Plourd)
    timepatch=time.time()-startpatch
    
    return costup, Plourd, tc, subnumb, lyc, chork



#check to see if solution is feasible
def Tourcheck(Trch,Tlen):
    check1=0
    check2=0
    Trr=Trch.copy()
    good=0
    if len(Trch)==len(Tlen):
        check1=1
    for i in range(len(Trch)):
        if i not in Trr:
            break
        Trr.remove(i)   
    if not Trr:
        check2=1   
    if check1 == 1 and check2 ==1:
        good=0
    else:
        print("not a tour")
        good=1
        
    return good
        
#calculate cost of tours/subtours
def cost(costmatt, routeee):
    sumr=0
    sumr=costmatt[routeee[len(routeee)-1]][routeee[0]]
    for i in range(len(routeee)-1):
        sumr += costmatt[routeee[i]][routeee[i+1]]
    return sumr

def costd(cost_mato, routo):
    sumro=0
    sumro=cost_mato[routo[len(routo)-1]][routo[0]]
    for i in range(len(routo)-1):
        sumro += cost_mato[routo[i]][routo[i+1]]
    return sumro 


#determines the edges that add flow to the network
#it is a fairly simple process, but has several lines due to special cases
def Jumps(ListBru,inDru):
    newLBru=[[-1 for i in range(len(ListBru))]]
    Listhold=[]
    reverses=[]
    revstor=[-1,-1]
    jumps=[]
    jumpstor=[-1,-1]
    indexhold=-1
    counteu=0
 
    if (inDru[len(inDru)-1]==inDru[0]+1 or (inDru[len(inDru)-1]==0 and inDru[0]==(len(ListBru)-1))):
            revstor[0]=inDru[len(inDru)-1]
            revstor[1]=inDru[0]
            reverses.append(revstor)
            revstor=[-1 for i in range(2)]
            jumpstor[0]=inDru[0]
            jumpstor[1]=inDru[1]
            jumps.append(jumpstor)
            jumpstor=[-1 for i in range(2)]        
            
    if inDru[len(inDru)-2]==inDru[len(inDru)-1]+1 or inDru[len(inDru)-2]==0:
            revstor[0]=inDru[len(inDru)-1]
            revstor[1]=inDru[len(inDru)-2]
            reverses.append(revstor)
            revstor=[-1 for i in range(2)]
            jumpstor[0]=inDru[len(inDru)-1]
            jumpstor[1]=inDru[0]
            jumps.append(jumpstor)
            jumpstor=[-1 for i in range(2)]
    
    for p in range(len(inDru)-2):
        if inDru[p]==inDru[p+1]+1 or (inDru[p]+len(ListBru)-1==inDru[p+1]):
            revstor[0]=inDru[p+1]
            revstor[1]=inDru[p]
            reverses.append(revstor)
            revstor=[-1 for i in range(2)]
            jumpstor[0]=inDru[p+1]
            jumpstor[1]=inDru[p+2]
            jumps.append(jumpstor)
            jumpstor=[-1 for i in range(2)]
            
            
    return jumps,reverses


#Forms a single tour by combining subtours
def Listy(rourt, Subby):

    ListS=[]
    Listfiny=[]
    for k in range(len(Subby)):
        for j in range(len(Subby[k])):
            ListS.append(rourt[0][Subby[k][j]])
        Listfiny.append(ListS)
        ListS=[]
    return Listfiny


#finds the patch between cycle1 and cycle2
#this is also fairly straightforward, but has several lines to account for several cases
def Patcher(cycle1,cycle2,cost_matPat):
        
    store2=math.inf
    patch=[]
    
    if (len(cycle1)==1):
        for i in range(len(cycle2)-1):
            store1=cost_matPat[cycle2[i]][cycle1[0]]+cost_matPat[cycle1[0]][cycle2[i+1]]-cost_matPat[cycle2[i]][cycle2[i+1]]
            if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle2[i+1])
                patch.append(cycle2[i])
                patch.append(cycle1[0])
        
    if (len(cycle2)==1):
        for i in range(len(cycle1)-1):
            store1=cost_matPat[cycle1[i]][cycle2[0]]+cost_matPat[cycle2[0]][cycle1[i+1]]-cost_matPat[cycle1[i]][cycle1[i+1]]
            if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[i+1])
                patch.append(cycle1[i])
                patch.append(cycle2[0])
        
    for i in range(len(cycle1)-1):
        for j in range(len(cycle2)-1):
            store1=cost_matPat[cycle1[i]][cycle2[j+1]]+cost_matPat[cycle2[j]][cycle1[i+1]]-cost_matPat[cycle1[i]][cycle1[i+1]]-cost_matPat[cycle2[j]][cycle2[j+1]]
            if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[i+1])
                patch.append(cycle1[i])
                patch.append(cycle2[j+1])
                patch.append(cycle2[j])
                                        
    for i in range(len(cycle1)-1):
        store1=cost_matPat[cycle1[i]][cycle2[0]]+cost_matPat[cycle2[len(cycle2)-1]][cycle1[i+1]]-cost_matPat[cycle1[i]][cycle1[i+1]]-cost_matPat[cycle2[len(cycle2)-1]][cycle2[0]]
        if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[i+1])
                patch.append(cycle1[i])
                patch.append(cycle2[0])
                patch.append(cycle2[len(cycle2)-1])
                
    for j in range(len(cycle2)-1):
        store1=cost_matPat[cycle1[len(cycle1)-1]][cycle2[j+1]]+cost_matPat[cycle2[j]][cycle1[0]]-cost_matPat[cycle1[len(cycle1)-1]][cycle1[0]]-cost_matPat[cycle2[j]][cycle2[j+1]]
        if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[0])
                patch.append(cycle1[len(cycle1)-1])
                patch.append(cycle2[j+1])
                patch.append(cycle2[j])
                
    store1=cost_matPat[cycle1[len(cycle1)-1]][cycle2[0]]+cost_matPat[cycle2[len(cycle2)-1]][cycle1[0]]-cost_matPat[cycle1[len(cycle1)-1]][cycle1[0]]-cost_matPat[cycle2[len(cycle2)-1]][cycle2[0]]
    if store1<store2:
        store2=store1
        patch.clear()
        patch.append(cycle1[0])
        patch.append(cycle1[len(cycle1)-1])
        patch.append(cycle2[0])
        patch.append(cycle2[len(cycle2)-1])
                
    return store2, patch

#used to actually patch together subtours
def Patched(cycle1pa, cycle2pa, patchd):
            
    newLpa=[]
    newS=[]
    newE=[]
    newM1=[]
    newM2=[]
    switchpa=0
    firstpa=-1
    countrpa=0
    
    if len(patchd)==3:
        #find piece 1 of tour/subtour
        if len(cycle1pa)==1:
            for i in range(len(cycle2pa)):
                newLpa.append(cycle2pa[i])
                if cycle2pa[i]==patchd[1]:
                    newLpa.append(cycle1pa[0])
            return newLpa
         #find piece 2 of tour/subtour          
        if len(cycle2pa)==1:
            for i in range(len(cycle1pa)):
                newLpa.append(cycle1pa[i])
                if cycle1pa[i]==patchd[1]:
                    newLpa.append(cycle2pa[0])
            return newLpa           
      
    #find piece 3 of tour/subtour (should be a single edge)
    for i in range(len(cycle1pa)):
        if switchpa==0:
            newS.append(cycle1pa[i])
            if cycle1pa[i]==patchd[1]:
                switchpa=1
                firstpa=1
        if switchpa==1:
            if(firstpa != 1):
                newE.append(cycle1pa[i])
            firstpa=0

    #find piece 4 of tour/subtour (should be a single edge)
    switchpa=0
    for i in range(len(cycle2pa)):
        if switchpa==0:
            newM1.append(cycle2pa[i])
            if cycle2pa[i]==patchd[3]:
                switchpa=1
                firstpa=1
        if switchpa==1:
            if(firstpa != 1):
                newM2.append(cycle2pa[i])
            firstpa=0
    #combine the four pieces        
    newLpa=newS+newM2+newM1+newE
    
    return newLpa

#function iteratively patches subtours until a final tour is formed
def Tourpr(ListFind,datad,Subtad):
    if len(ListFind) > 1:
        costPat,patchPat= Patcher(ListFind[0],ListFind[1],datad)
        Patour= Patched(ListFind[0], ListFind[1], patchPat)
        if len(ListFind) > 1:
            for j in range(len(Subtad)-2):
                costPat,patchPat = Patcher(Patour,ListFind[j+2],datad)
                Patour=Patched(Patour,ListFind[j+2],patchPat)      
    return Patour


#The majority of the following code is contributed by Pranchalk  
#and found on www.geeksforgeeks.org/ I made some adjustments for it
#to work in conjunction in our code
def minmean(dart,routy):
    
    iindy=2
    routerindy = [[-1 for i in range(len(dart))] for j in range(iindy)]
    
    #initialize a 2-d array for indices and vertices
    for i in range(iindy):
        for j in range(len(dart)):
            routerindy[0][j]=routy[j]
            routerindy[1][j]=j
    
    # a struct to represent edges
    class edge:
        def __init__(self, u, w):
            self.From = u
            self.weight = w
 
    def addedge(u, v, w):
        edges[v].append(edge(u, w))
 
    # calculates the shortest path
    def shortestpath(dp,pp):       
        maxi=1e8
        # initializing all distances as -1
        for i in range(V + 1):
            for j in range(V):
                dp[i][j] = maxi
                pp[i][j] = -1
 
        # shortest distance From first vertex
        # to in itself consisting of 0 edges
        dp[0][0] = 0
 
        # filling up the dp table
        for i in range(1, V + 1):
            for j in range(V):
                for k in range(len(edges[j])):
                    if (dp[i - 1][edges[j][k].From] != int(maxi)):
                        curr_wt = (dp[i - 1][edges[j][k].From] + edges[j][k].weight)
            
                        if (dp[i][j] == int(maxi)):
                            dp[i][j] = curr_wt
                            if(curr_wt<(maxi-50000)):
                                pp[i][j]=edges[j][k].From
                        else:
                            dp[i][j] = min(dp[i][j], curr_wt)
                            if(dp[i][j] == curr_wt and curr_wt<(maxi-50000)):
                                pp[i][j]=edges[j][k].From

 
    # Returns minimum value of average
    # weight of a cycle in graph.
    def minAvgWeight():
        dp = [[None] * V for i in range(V + 1)]
        pp = [[None] * V for i in range(V + 1)]
        shortestpath(dp,pp)
 
        # array to store the avg values
        avg = [-maxi] * V
 
        # Compute average values for all
        # vertices using weights of
        # shortest paths store in dp.
        for i in range(V):
            if (dp[V][i] != maxi):
                for j in range(V):
                    if (dp[j][i] < maxi-1):
                        if(avg[i]<(dp[V][i] - dp[j][i]) / (V - j) and (dp[V][i] - dp[j][i]) / (V - j) < 0):
                            avg[i] = max(avg[i], (dp[V][i] - dp[j][i]) / (V - j))
 
        # Find minimum value in avg[]
        result = avg[0]
        story=0
        for i in range(V):
            if (avg[i] > -maxi/2 and avg[i] < result):
                result = avg[i]
                story=i
                
                
        return result, pp, story
 
# Driver Code

    
    novert = len(dart[0])
    holddat=dart
    V = 2 * novert
    maxi=1e8
    
    edges = [[] for i in range(V)]

    route=routy
    
    for i in range(len(route)-1):  
        holddat[route[i]][route[i+1]]=maxi
        holddat[route[i+1]][route[i]]=-holddat[route[i+1]][route[i]]
    
    holddat[route[0]][route[len(route)-1]]=-holddat[route[0]][route[len(route)-1]]
    holddat[route[len(route)-1]][route[0]]=maxi

    for i in range(int(V/2)):
        for j in range(int(V)): 
            if(i<novert and j>=novert):
                if(i == (j-novert) or holddat[i][j-novert]<0):
                    addedge(i,j,maxi)
                else: 
                    addedge(i,j,holddat[i][j-novert])
            else:
                addedge(i,j,maxi)
        
    for i in range(int(V/2)):
        for j in range(int(V)):
            if(j<novert):
                if(holddat[i][j]<0):
                    addedge(i+novert,j,holddat[i][j])
                else:
                    addedge(i+novert,j,maxi)
            else:
                addedge(i+novert,j,maxi)
                


    def Cyclef(porpe,storie,Ve):
    
        store = [storie]
        store.append(porpe[Ve][storie])

        for i in range(Ve-1):
            store.append(porpe[Ve-i-1][store[len(store)-1]])
        newlist = [] 
        doubly = 0
        for i in store:
            if i not in newlist:
                newlist.append(i)
            else:
                doubly=i # this method catches the first duplicate entries, and appends them to the list
                break
    
        store2=[]
        k=1

        for j in range(len(store)):
            if (store[j]==doubly):
                store2.append(doubly)
                while (store[j+k]!=doubly):
                    store2.append(store[j+k])
                    k=k+1
                break
            
        return store2
    

    resort, porp, stori = minAvgWeight()
    

    cyclee=Cyclef(porp,stori,2*len(dart[0]))
    sumr=edges[cyclee[len(cyclee)-1]][cyclee[0]].weight
    
    for i in range(len(cyclee)-1):
        sumr+=edges[cyclee[i]][cyclee[i+1]].weight
        
    for k in range(len(cyclee)):
        if cyclee[k]>(len(dart)-1):
            cyclee[k]=cyclee[k]-(len(dart))
    
    return cyclee, sumr, routerindy
            




#The remaining functions where either entirely not written by me, or adjusted by me

def two_opt(cost_matrt,routrt):
    routrhold=[]
    routrhold=routrt
    best = routrt.copy()
    improved = True
    while improved:
        improved = False
        for i in range(1, len(routrt)-2):
            for j in range(i+1, len(routrt)+1):
                if j-i == 1: continue # changes nothing, skip then
                new_route = routrt.copy()
                new_route[i:j] = routrt[j-1:i-1:-1] # this is the 2woptSwap
                if cost(cost_matrt, new_route) < cost(cost_matrt, best):
                    best=new_route
                    improved = True
        routrt = best
    if cost(cost_matrt, best) > cost(cost_matrt, routrhold):
        return routrhold, cost(cost_matrt, routrhold)
    else:
        return best, cost(cost_matrt, best)

class OptCase(Enum):
    opt_case_1 = "opt_case_1"
    opt_case_2 = "opt_case_2"
    opt_case_3 = "opt_case_3"
    opt_case_4 = "opt_case_4"
    opt_case_5 = "opt_case_5"
    opt_case_6 = "opt_case_6"
    opt_case_7 = "opt_case_7"
    opt_case_8 = "opt_case_8"
    
    

def tsp_3_opt(graph, route):
    import time
    timeout = time.time() + 1000
    lengty=len(graph)
    moves_cost = {OptCase.opt_case_1: 0, OptCase.opt_case_2: 0,
                  OptCase.opt_case_3: 0, OptCase.opt_case_4: 0, OptCase.opt_case_5: 0,
                  OptCase.opt_case_6: 0, OptCase.opt_case_7: 0, OptCase.opt_case_8: 0}
    improved = True
    best_found_route = route
    besty_route = route
    stored=cost(graph,best_found_route)
    timeout_start = time.time()
    while improved:
        improved = False
        for (i, j, k) in possible_segments(len(graph)):
            time27 = time.time()
            if time27 > timeout:
                return besty_route
#             # we check all the possible moves and save the result into the dict
            if(cost(graph,best_found_route)<stored):
                besty_route=best_found_route
            for opt_case in OptCase:
                moves_cost[opt_case] = get_solution_cost_change(graph, best_found_route, opt_case, i, j, k)
            # we need the minimum value of substraction of old route - new route
            best_return = max(moves_cost, key=moves_cost.get)
            if moves_cost[best_return] > 0:
                best_found_route = reverse_segments(best_found_route, best_return, i, j, k)
                improved = True
                break
    # just to start with the same node -> we will need to cycle the results.
    cycled = cycle(best_found_route)
    skipped = dropwhile(lambda x: x != 0, cycled)
    sliced = islice(skipped, None, len(best_found_route))
    best_found_route = list(sliced)
    return best_found_route


def possible_segments(N):
    """ Generate the combination of segments """
    segments = ((i, j, k) for i in range(N) for j in range(i + 2, N-1) for k in range(j + 2, N - 1 + (i > 0)))
    return segments


def get_solution_cost_change(graph, route, case, i, j, k):
    """ Compare current solution with 7 possible 3-opt moves"""
    A, B, C, D, E, F = route[i - 1], route[i], route[j - 1], route[j], route[k - 1], route[k % len(route)]
    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        return 0
    elif case == OptCase.opt_case_2:
        # second case is the case A'BC
        return graph[A, B] + graph[E, F] - (graph[B, F] + graph[A, E])
    elif case == OptCase.opt_case_3:
        # ABC'
        return graph[C, D] + graph[E, F] - (graph[D, F] + graph[C, E])
    elif case == OptCase.opt_case_4:
        # A'BC'
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[A, D] + graph[B, F] + graph[E, C])
    elif case == OptCase.opt_case_5:
        # A'B'C
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[C, F] + graph[B, D] + graph[E, A])
    elif case == OptCase.opt_case_6:
        # AB'C
        return graph[B, A] + graph[D, C] - (graph[C, A] + graph[B, D])
    elif case == OptCase.opt_case_7:
        # AB'C'
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[B, E] + graph[D, F] + graph[C, A])
    elif case == OptCase.opt_case_8:
        # A'B'C
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[A, D] + graph[C, F] + graph[B, E])

def reverse_segments(route, case, i, j, k):
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]

    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        pass
    elif case == OptCase.opt_case_2:
        # A'BC
        solution = list(reversed(first_segment)) + second_segment + third_segment
    elif case == OptCase.opt_case_3:
        # ABC'
        solution = first_segment + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_4:
        # A'BC'
        solution = list(reversed(first_segment)) + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_5:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_6:
        # AB'C
        solution = first_segment + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_7:
        # AB'C'
        solution = first_segment + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_8:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment))
    return solution




# Updated Opts

In [ ]:
def two_opt(cost_matrt,routrt):
    routrhold=[]
    routre=[]
    routrhold=np.copy(routrt)
    best = routrt.copy()
    improved = True
    while improved:
        improved = False
        for i in range(1, len(routrt)-2):
            for j in range(i+1, len(routrt)+1):
                if j-i == 1: continue # changes nothing, skip then
                new_route = routrt.copy()
                new_route[i:j] = routrt[j-1:i-1:-1] # this is the 2woptSwap
                routre=list(reversed(new_route))
                if cost(cost_matrt, routre) < cost(cost_matrt, new_route):
                    new_route=routre
                if cost(cost_matrt, new_route) < cost(cost_matrt, best):
                    best=new_route
                    improved = True
        routrt = best
        
    #print("best",best,"hold",routrhold)
    if cost(cost_matrt, best) > cost(cost_matrt, routrhold):
        return routrhold, cost(cost_matrt, routrhold)
    else:
        return best, cost(cost_matrt, best)




class OptCase(Enum):
    opt_case_1 = "opt_case_1"
    opt_case_2 = "opt_case_2"
    opt_case_3 = "opt_case_3"
    opt_case_4 = "opt_case_4"
    opt_case_5 = "opt_case_5"
    opt_case_6 = "opt_case_6"
    opt_case_7 = "opt_case_7"
    opt_case_8 = "opt_case_8"
    opt_case_9 = "opt_case_9"
    opt_case_10 = "opt_case_10"
    opt_case_11 = "opt_case_11"
    opt_case_12 = "opt_case_12"
    opt_case_13 = "opt_case_13"
    opt_case_14 = "opt_case_14"
    opt_case_15 = "opt_case_15"
    opt_case_16 = "opt_case_16"
    
    

def tsp_3_opt(graph, route):
#     routeup=np.array(route)
    routeup=route
    route=[0 for i in range(len(routeup))]
    for i in range(len(routeup)):
        route[i]=routeup[i]
    import time
    #The following variable was included by me because this 3-opt can get stuck in an infinite loop
    timeout = time.time() + 2
    lengty=len(graph)
    moves_cost = {OptCase.opt_case_1: 0, OptCase.opt_case_2: 0,
                  OptCase.opt_case_3: 0, OptCase.opt_case_4: 0, OptCase.opt_case_5: 0,
                  OptCase.opt_case_6: 0, OptCase.opt_case_7: 0, OptCase.opt_case_8: 0,
                  OptCase.opt_case_9: 0, OptCase.opt_case_10: 0,
                  OptCase.opt_case_11: 0, OptCase.opt_case_12: 0, OptCase.opt_case_13: 0,
                  OptCase.opt_case_14: 0, OptCase.opt_case_15: 0, OptCase.opt_case_16: 0}
    improved = True
    best_found_route = route
    besty_route = route
    stored=cost(graph,best_found_route)
    timeout_start = time.time()
    while improved:
        improved = False
        for (i, j, k) in possible_segments(len(graph)):
            time27 = time.time()
            if time27 > timeout:
                return besty_route
            if(cost(graph,best_found_route)<stored):
                besty_route=best_found_route
            for opt_case in OptCase:
                moves_cost[opt_case] = get_solution_cost_change(graph, best_found_route, opt_case, i, j, k)
            # we need the minimum value of substraction of old route - new route
            best_return = max(moves_cost, key=moves_cost.get)
            if moves_cost[best_return] > 0:
#                 print("")
#                 print("best",best_found_route, cost(danew,best_found_route))
#                 print("")
                best_found_route = reverse_segments(best_found_route, best_return, i, j, k)
                improved = True
                break
    # just to start with the same node -> we will need to cycle the results.
    cycled = cycle(best_found_route)
    skipped = dropwhile(lambda x: x != 0, cycled)
    sliced = islice(skipped, None, len(best_found_route))
    best_found_route = list(sliced)
    return best_found_route


def possible_segments(N):
    """ Generate the combination of segments """
    segments = ((i, j, k) for i in range(N) for j in range(i + 2, N-1) for k in range(j + 2, N - 1 + (i > 0)))
    return segments


def get_solution_cost_change(graph, route, case, i, j, k):
    """ Compare current solution with 7 possible 3-opt moves"""
    A, B, C, D, E, F = route[i - 1], route[i], route[j - 1], route[j], route[k - 1], route[k % len(route)]
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]
    
    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        return 0
    elif case == OptCase.opt_case_2:
        # second case is the case A'BC
        return -cost(graph,list(reversed(first_segment)) + second_segment + third_segment)+cost(graph,route)
    elif case == OptCase.opt_case_3:
        # ABC'
        return -cost(graph,first_segment + second_segment + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_4:
        # A'BC'
        return -cost(graph,list(reversed(first_segment)) + second_segment + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_5:
        # A'B'C
        return -cost(graph,list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment)+cost(graph,route)
    elif case == OptCase.opt_case_6:
        # AB'C
        return -cost(graph,first_segment + list(reversed(second_segment)) + third_segment)+cost(graph,route)
    elif case == OptCase.opt_case_7:
        # AB'C'
        return -cost(graph,first_segment + list(reversed(second_segment)) + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_8:
        # A'B'C
        return -cost(graph,list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_9:
        # second case is the case A'BC
        return -cost(graph,list(reversed(third_segment)) + list(reversed(second_segment)) + first_segment)+cost(graph,route)
    elif case == OptCase.opt_case_10:
        # ABC'
        return -cost(graph,third_segment + list(reversed(second_segment)) + list(reversed(first_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_11:
        # A'BC'
        return -cost(graph,third_segment + second_segment + list(reversed(first_segment)))+cost(graph,route) 
    elif case == OptCase.opt_case_12:
        # A'B'C
        return -cost(graph,third_segment + second_segment + list(reversed(first_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_13:
        # AB'C
        return -cost(graph,list(reversed(third_segment)) + second_segment + list(reversed(first_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_14:
        # AB'C'
        return -cost(graph,list(reversed(third_segment)) + second_segment + first_segment)+cost(graph,route)
    elif case == OptCase.opt_case_15:
        # A'B'C
        return -cost(graph,third_segment + second_segment + first_segment)+cost(graph,route)
    elif case == OptCase.opt_case_16:
        # A'B'C
        return -cost(graph,list(reversed(third_segment)) + list(reversed(second_segment)) + list(reversed(first_segment)))+cost(graph,route)

    

def reverse_segments(route, case, i, j, k):
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]

    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        pass
    elif case == OptCase.opt_case_2:
        # A'BC
        solution = list(reversed(first_segment)) + second_segment + third_segment
        
    elif case == OptCase.opt_case_3:
        # ABC'
        solution = first_segment + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_4:
        # A'BC'
        solution = list(reversed(first_segment)) + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_5:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_6:
        # AB'C
        solution = first_segment + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_7:
        # AB'C'
        solution = first_segment + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_8:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_9:
        # A'BC
        solution = list(reversed(third_segment)) + list(reversed(second_segment)) + first_segment
    elif case == OptCase.opt_case_10:
        # ABC'
        solution = third_segment + list(reversed(second_segment)) + list(reversed(first_segment))
    elif case == OptCase.opt_case_11:
        # A'BC'
        solution = third_segment + list(reversed(second_segment)) + first_segment
    elif case == OptCase.opt_case_12:
        # A'B'C
        solution = third_segment + second_segment + list(reversed(first_segment))
    elif case == OptCase.opt_case_13:
        # AB'C
        solution = list(reversed(third_segment)) + second_segment + list(reversed(first_segment))
    elif case == OptCase.opt_case_14:
        # AB'C'
        solution = list(reversed(third_segment)) + second_segment + first_segment
    elif case == OptCase.opt_case_15:
        # A'B'C
        solution = third_segment + second_segment + first_segment
    elif case == OptCase.opt_case_16:
        # A'B'C
        solution = list(reversed(third_segment)) + list(reversed(second_segment)) + list(reversed(first_segment))
    
    #print("sol",solution,cost(danew,solution))
    return solution